In [1]:
import plotly.graph_objects as go

from datetime import datetime
import sys

import pandas as pd


In [2]:
# Ralative imports
# Add the src directory to the Python path
sys.path.insert(0, '../src')

from botcoin.data.historical import YfDataManager, TimeGranularity
from botcoin.utils.stream_data import generate_price_stream

In [3]:
hdm = YfDataManager()

# Create a naive datetime
start = datetime(year=2025, month=5, day=30, hour=0, minute=0, second=0)
end = datetime(year=2025, month=6, day=1, hour=0, minute=0, second=0)

start   = hdm.tz.localize(start)
end     = hdm.tz.localize(end)

symbol = "AAPL"

In [4]:
hdm.get_current_data_date_range(symbol, TimeGranularity.ONE_MINUTE)

(datetime.date(2025, 5, 30), datetime.date(2025, 6, 3))

In [5]:
df = hdm.get_ohlcv_1min("AAPL", start_date=start.date(), end_date=end.date())

2025-06-02 22:47:06,048 - DataManager - DEBUG: Retrieving data from local storage.


In [6]:
df.head()

,Close,High,Low,Open,Volume
Datetime,,,,,
2025-05-30 09:30:00-04:00,198.889999,199.570007,198.750000,199.404999,2371867
2025-05-30 09:31:00-04:00,199.169998,199.210007,198.699997,198.889999,192407
2025-05-30 09:32:00-04:00,199.210007,199.347397,199.089996,199.130005,68929
2025-05-30 09:33:00-04:00,198.884995,199.070007,198.729996,198.800003,258685
2025-05-30 09:34:00-04:00,198.970001,199.080002,198.530396,198.875000,159008


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 388 entries, 2025-05-30 09:30:00-04:00 to 2025-05-30 15:59:00-04:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   388 non-null    float64
 1   High    388 non-null    float64
 2   Low     388 non-null    float64
 3   Open    388 non-null    float64
 4   Volume  388 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 18.2 KB


In [8]:
df.index.tz

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

In [9]:
prices = generate_price_stream(df, candle_duration='1min', avg_freq_per_minute=10)

In [10]:
prices.head()

,price
timestamp,
1.748612e+09,199.404999
1.748612e+09,199.570007
1.748612e+09,198.750000
1.748612e+09,198.889999
1.748612e+09,199.082221


In [11]:
# Convert the timestamp index to a datetime object
prices.index = pd.to_datetime(prices.index, unit='s')
prices.index = prices.index.tz_localize('UTC')             # mark them as UTC
prices.index = prices.index.tz_convert('US/Eastern')       # convert to US Eastern

In [12]:
prices.index.tz

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

In [13]:
prices.head()

,price
timestamp,
2025-05-30 09:30:29.300962925-04:00,199.404999
2025-05-30 09:30:31.513391972-04:00,199.570007
2025-05-30 09:30:40.806324005-04:00,198.750000
2025-05-30 09:30:53.115580082-04:00,198.889999
2025-05-30 09:30:55.421340942-04:00,199.082221


In [14]:
def plot_candles_with_price_stream(ohlc_df, price_stream_df):
    """
    Plot OHLC candlesticks and overlay price stream using Plotly.
    
    Args:
        ohlc_df (pd.DataFrame): Must have DatetimeIndex and columns ['Open', 'High', 'Low', 'Close'].
        price_stream_df (pd.DataFrame): Must have DatetimeIndex and column ['price'].
    """
    fig = go.Figure()

    # Candlestick chart
    fig.add_trace(go.Candlestick(
        x=ohlc_df.index,
        open=ohlc_df['Open'],
        high=ohlc_df['High'],
        low=ohlc_df['Low'],
        close=ohlc_df['Close'],
        name='Candles',
        increasing_line_color='green',
        decreasing_line_color='red'
    ))

    # Simulated price points as a line
    fig.add_trace(go.Scatter(
        x=price_stream_df.index,
        y=price_stream_df['price'],
        mode='lines+markers',
        name='Simulated Prices',
        line=dict(color='blue', width=1),
        marker=dict(size=4)
    ))

    # Styling
    fig.update_layout(
        title="OHLC Candles with Simulated Tick Data",
        xaxis_title="Time",
        yaxis_title="Price",
        xaxis_rangeslider_visible=False,
        template='plotly_dark',
        height=600,
        width=1000
    )

    fig.show()

In [15]:
plot_candles_with_price_stream(df, prices)